In [1]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.
c:\users\rahul\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\rahul\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\rahul\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [2]:
def load_doc(filename):    
    file= open('results.csv','r',encoding='utf-8')
    text=file.read()
    file.close()
    return text

In [3]:
def load_descriptions():
    mapping=dict()
    text=load_doc('results.csv')
    for line in text.split('\n'):
        tokens=line.split('|')
        #print(tokens[0]," ",tokens[1:],end='\n')
        image_id,image_desc=tokens[0],tokens[-1]
        image_id=image_id.split('.')[0]
        
        if image_id not in mapping:
            mapping[image_id]=list()
        mapping[image_id].append(image_desc)
        
    return mapping

In [4]:
descriptions=load_descriptions()

In [5]:
list(descriptions.keys())[:5]

['image_name', '1000092795', '10002456', '1000268201', '1000344755']

In [6]:
descriptions['1000268201']

[' A child in a pink dress is climbing up a set of stairs in an entry way .',
 ' A little girl in a pink dress going into a wooden cabin .',
 ' A little girl climbing the stairs to her playhouse .',
 ' A little girl climbing into a wooden playhouse ',
 ' A girl going into a wooden building .']

In [7]:
descriptions['1000344755']

[' Someone in a blue shirt and hat is standing on stair and leaning against a window .',
 ' A man in a blue shirt is standing on a ladder cleaning a window .',
 ' A man on a ladder cleans the window of a tall building .',
 ' man in blue shirt and jeans on ladder cleaning windows',
 ' a man on a ladder cleans a window']

In [8]:
def clean_descriptions(descriptions):
    table=str.maketrans('','',string.punctuation)
    for key,desc_list in descriptions.items():
        for i in range(len(desc_list)):
            desc=desc_list[i]
            desc=desc.split()
            desc=[word.lower() for word in desc]
            desc=[w.translate(table) for w in desc]
            desc=[word for word in desc if len(word)>1]
            desc=[word for word in desc if word.isalpha()]
            desc_list[i]=' '.join(desc)


In [9]:
clean_descriptions(descriptions)

In [10]:
descriptions['1000344755']

['someone in blue shirt and hat is standing on stair and leaning against window',
 'man in blue shirt is standing on ladder cleaning window',
 'man on ladder cleans the window of tall building',
 'man in blue shirt and jeans on ladder cleaning windows',
 'man on ladder cleans window']

In [11]:
descriptions['1000268201']

['child in pink dress is climbing up set of stairs in an entry way',
 'little girl in pink dress going into wooden cabin',
 'little girl climbing the stairs to her playhouse',
 'little girl climbing into wooden playhouse',
 'girl going into wooden building']

In [13]:
def save_descriptions(descriptions,filename):
    lines=list()
    for key,desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key+' '+desc)
    data='\n'.join(lines)
    
    file=open(filename,'w')
    file.write(data)
    file.close()
    

In [14]:
all_desc=set()
for key in descriptions.keys():
    #print(key)
    for d in descriptions[key]:
        [all_desc.update(d.split())]

In [15]:
vocabulary=all_desc
print("size of the vocabulary=",len(vocabulary))

size of the vocabulary= 19736


In [16]:
def load_set(filename):
    doc=load_doc(filename)
    dataset=list()
    for line in doc.split('\n'):
        if len(line)<1:
            continue
        identifier=line.split('.')[0]
        dataset.append(identifier)
    return set(dataset)


In [17]:
filename='results.csv'
train=load_set(filename)
print('dataset=',len(train))

dataset= 31784


In [18]:
images='./flickr30k_images/flickr30k_images/'
img=glob.glob(images+'*.jpg')

In [19]:
def preprocess(image_path):
    img=image.load_img(image_path,target_size=(299,299))
    x=image.img_to_array(img)
    x=np.expand_dims(x,axis=0)
    x=preprocess_input(x)
    return x

In [20]:
model=InceptionV3(weights='imagenet')

In [21]:
model_new=Model(model.input,model.layers[-2].output)


In [22]:
def encode(image):
    image=preprocess(image)
    fea_vec=model_new.predict(image)
    fea_vec=np.reshape(fea_vec,fea_vec.shape[1])
    return fea_vec

In [47]:
start=time()
encoding_train={}
for img in img:
    encoding_train[img[len(images):]]=encode(img)
    print("time tken in second=",time()-start)



time tken in second= 14.423187017440796
time tken in second= 14.527613639831543
time tken in second= 14.606151819229126
time tken in second= 14.695779085159302
time tken in second= 14.785825729370117
time tken in second= 14.861010074615479
time tken in second= 14.916728019714355
time tken in second= 14.966171503067017
time tken in second= 15.034387111663818
time tken in second= 15.090407848358154
time tken in second= 15.167502164840698
time tken in second= 15.256842374801636
time tken in second= 15.32330060005188
time tken in second= 15.399495601654053
time tken in second= 15.489752054214478
time tken in second= 15.550183296203613
time tken in second= 15.636332035064697
time tken in second= 15.702350616455078
time tken in second= 15.759394884109497
time tken in second= 15.826125144958496
time tken in second= 15.882017374038696
time tken in second= 15.938784837722778
time tken in second= 15.995093584060669
time tken in second= 16.054452419281006
time tken in second= 16.12291646003723
t

time tken in second= 28.28032112121582
time tken in second= 28.35575008392334
time tken in second= 28.411210298538208
time tken in second= 28.47739553451538
time tken in second= 28.55305314064026
time tken in second= 28.625171661376953
time tken in second= 28.681041479110718
time tken in second= 28.759543657302856
time tken in second= 28.832947969436646
time tken in second= 28.8937087059021
time tken in second= 28.948886156082153
time tken in second= 29.02630853652954
time tken in second= 29.101784229278564
time tken in second= 29.15675711631775
time tken in second= 29.209349632263184
time tken in second= 29.285077571868896
time tken in second= 29.344033002853394
time tken in second= 29.416438579559326
time tken in second= 29.487207651138306
time tken in second= 29.55513286590576
time tken in second= 29.630763053894043
time tken in second= 29.71305227279663
time tken in second= 29.77752923965454
time tken in second= 29.851625680923462
time tken in second= 29.920684814453125
time tken i

time tken in second= 42.14721083641052
time tken in second= 42.20695161819458
time tken in second= 42.28721499443054
time tken in second= 42.34420442581177
time tken in second= 42.41407370567322
time tken in second= 42.47809076309204
time tken in second= 42.53800940513611
time tken in second= 42.61674094200134
time tken in second= 42.69390511512756
time tken in second= 42.75558519363403
time tken in second= 42.825875759124756
time tken in second= 42.90774321556091
time tken in second= 42.98190116882324
time tken in second= 43.054564237594604
time tken in second= 43.1122522354126
time tken in second= 43.18056559562683
time tken in second= 43.233182191848755
time tken in second= 43.30455732345581
time tken in second= 43.363085985183716
time tken in second= 43.42785406112671
time tken in second= 43.502031326293945
time tken in second= 43.56057548522949
time tken in second= 43.628031969070435
time tken in second= 43.686012744903564
time tken in second= 43.74457764625549
time tken in second

time tken in second= 56.10939288139343
time tken in second= 56.166818380355835
time tken in second= 56.23070049285889
time tken in second= 56.29647088050842
time tken in second= 56.36019992828369
time tken in second= 56.423335790634155
time tken in second= 56.491289138793945
time tken in second= 56.55977916717529
time tken in second= 56.619464635849
time tken in second= 56.69913101196289
time tken in second= 56.761855125427246
time tken in second= 56.8269739151001
time tken in second= 56.883997201919556
time tken in second= 56.94340753555298
time tken in second= 56.99922966957092
time tken in second= 57.064595460891724
time tken in second= 57.13214731216431
time tken in second= 57.188260555267334
time tken in second= 57.26627278327942
time tken in second= 57.32732677459717
time tken in second= 57.38639712333679
time tken in second= 57.44244194030762
time tken in second= 57.51201629638672
time tken in second= 57.57936215400696
time tken in second= 57.65514636039734
time tken in second= 

time tken in second= 69.63387393951416
time tken in second= 69.69966101646423
time tken in second= 69.76767539978027
time tken in second= 69.84149026870728
time tken in second= 69.91108703613281
time tken in second= 69.96967649459839
time tken in second= 70.03781509399414
time tken in second= 70.11701560020447
time tken in second= 70.19357514381409
time tken in second= 70.25085425376892
time tken in second= 70.34133243560791
time tken in second= 70.40560603141785
time tken in second= 70.47299933433533
time tken in second= 70.53370237350464
time tken in second= 70.60019731521606
time tken in second= 70.69106650352478
time tken in second= 70.77751612663269
time tken in second= 70.83885478973389
time tken in second= 70.90392422676086
time tken in second= 70.97982573509216
time tken in second= 71.07366228103638
time tken in second= 71.13528561592102
time tken in second= 71.19294857978821
time tken in second= 71.2393524646759
time tken in second= 71.29028248786926
time tken in second= 71.34

...

time tken in second= 1653.0705358982086
time tken in second= 1653.108468055725
time tken in second= 1653.1464240550995
time tken in second= 1653.1924557685852
time tken in second= 1653.251216173172
time tken in second= 1653.3046123981476
time tken in second= 1653.3595695495605
time tken in second= 1653.3995320796967
time tken in second= 1653.4429919719696
time tken in second= 1653.4839611053467
time tken in second= 1653.527953863144
time tken in second= 1653.583699464798
time tken in second= 1653.653122663498
time tken in second= 1653.7058265209198
time tken in second= 1653.7579839229584
time tken in second= 1653.7948758602142
time tken in second= 1653.8445477485657
time tken in second= 1653.8939113616943
time tken in second= 1653.9406316280365
time tken in second= 1653.9974102973938
time tken in second= 1654.0422852039337
time tken in second= 1654.0801491737366
time tken in second= 1654.130346775055
time tken in second= 1654.1807811260223
time tken in second= 1654.2296504974365
time t

time tken in second= 1663.2334241867065
time tken in second= 1663.278954744339
time tken in second= 1663.3236804008484
time tken in second= 1663.3894336223602
time tken in second= 1663.461983680725
time tken in second= 1663.5293769836426
time tken in second= 1663.5792844295502
time tken in second= 1663.6178712844849
time tken in second= 1663.6641821861267
time tken in second= 1663.7090182304382
time tken in second= 1663.7488460540771
time tken in second= 1663.7987904548645
time tken in second= 1663.8366878032684
time tken in second= 1663.873622894287
time tken in second= 1663.9265973567963
time tken in second= 1663.9706780910492
time tken in second= 1664.021852016449
time tken in second= 1664.0597422122955
time tken in second= 1664.1088814735413
time tken in second= 1664.1523213386536
time tken in second= 1664.2097635269165
time tken in second= 1664.2875545024872
time tken in second= 1664.332349538803
time tken in second= 1664.3857598304749
time tken in second= 1664.440214395523
time t

time tken in second= 1673.8338754177094
time tken in second= 1673.8963069915771
time tken in second= 1673.9549160003662
time tken in second= 1674.0150508880615
time tken in second= 1674.060409784317
time tken in second= 1674.1159732341766
time tken in second= 1674.1528570652008
time tken in second= 1674.205914258957
time tken in second= 1674.260767698288
time tken in second= 1674.3135714530945
time tken in second= 1674.3761706352234
time tken in second= 1674.4253115653992
time tken in second= 1674.4827013015747
time tken in second= 1674.531401872635
time tken in second= 1674.5816533565521
time tken in second= 1674.6308829784393
time tken in second= 1674.6810739040375
time tken in second= 1674.7204933166504
time tken in second= 1674.768383026123
time tken in second= 1674.8114085197449
time tken in second= 1674.8555765151978
time tken in second= 1674.898497581482
time tken in second= 1674.9678030014038
time tken in second= 1675.0249173641205
time tken in second= 1675.0725746154785
time t

time tken in second= 1684.3920094966888
time tken in second= 1684.4538373947144
time tken in second= 1684.4957447052002
time tken in second= 1684.5441880226135
time tken in second= 1684.58908867836
time tken in second= 1684.642956495285
time tken in second= 1684.686322927475
time tken in second= 1684.744461774826
time tken in second= 1684.835043668747
time tken in second= 1684.8838708400726
time tken in second= 1684.9312016963959
time tken in second= 1684.9790334701538
time tken in second= 1685.017948627472
time tken in second= 1685.06201171875
time tken in second= 1685.1051814556122
time tken in second= 1685.1526091098785
time tken in second= 1685.1915044784546
time tken in second= 1685.2488822937012
time tken in second= 1685.2927830219269
time tken in second= 1685.3466820716858
time tken in second= 1685.419471502304
time tken in second= 1685.471731185913
time tken in second= 1685.524112701416
time tken in second= 1685.5630133152008
time tken in second= 1685.61181807518
time tken in s

time tken in second= 1695.0925583839417
time tken in second= 1695.1314532756805
time tken in second= 1695.1693665981293
time tken in second= 1695.2215883731842
time tken in second= 1695.3023583889008
time tken in second= 1695.3572361469269
time tken in second= 1695.4097924232483
time tken in second= 1695.491332769394
time tken in second= 1695.552167892456
time tken in second= 1695.5950682163239
time tken in second= 1695.6391382217407
time tken in second= 1695.7005643844604
time tken in second= 1695.7443549633026
time tken in second= 1695.7852528095245
time tken in second= 1695.8388240337372
time tken in second= 1695.8756968975067
time tken in second= 1695.925544500351
time tken in second= 1695.9697804450989
time tken in second= 1696.013685464859
time tken in second= 1696.0618171691895
time tken in second= 1696.112323999405
time tken in second= 1696.154176235199
time tken in second= 1696.1973052024841
time tken in second= 1696.2494640350342
time tken in second= 1696.3085088729858
time t

In [50]:
with open('encoded_train_images.pkl','wb') as f:
    dump(encoding_train,f)

In [23]:
train_features=load(open('encoded_train_images.pkl','rb'))
print(len(train_features))

31783


In [24]:
all_train_captions=[]
for key,val in descriptions.items():
    for cap in val:
        all_train_captions.append(cap)
len(all_train_captions)

158917

In [25]:
word_count_threshold=10
word_counts={}
nsents=0
for sent in all_train_captions:
    nsents+=1
    for w in sent.split(' '):
        word_counts[w]=word_counts.get(w,0)+1
        
vocab=[w for w in word_counts if word_counts[w]>=word_count_threshold]
print("no of word =",len(word_counts))
print("len of vocabulary=",len(vocab))

no of word = 19737
len of vocabulary= 5435


In [26]:
ixtoword={}
wordtoix={}
ix=1
for w in vocab:
    wordtoix[w]=ix
    ixtoword[ix]=w
    ix+=1
    

In [27]:
with open('ixtoword.pkl','wb') as f1:
    dump(ixtoword,f1)
    f1.close()

with open('wordtoix.pkl','wb') as f2:
    dump(wordtoix,f2)
    f2.close()


In [28]:
vocab_size=len(ixtoword)+1
vocab_size

5436

In [29]:
def to_lines(descriptions):
    all_desc=list()
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

In [30]:
print(len(to_lines(descriptions)))

158917


In [31]:
def max_length(descriptions):
    lines=to_lines(descriptions)
    return max(len(d.split()) for d in lines)

In [32]:
max_length=max_length(descriptions)


In [33]:
print("length of max length descriptions=",max_length)

length of max length descriptions= 72


In [34]:
def data_generator(descriptions,photos,wordtoix,max_length,num_photos_per_batch):
    X1,X2,y=list(),list(),list()
    n=0
    
    while 1:
        for key,desc_list in descriptions.items():
            n+=1
            photo=photos[key+'.jpg']
            for desc in desc_list:
                seq=[wordtoix[word] for word in desc.split(' ') if word in wordtoix ]
                for i in range(1,len(seq)):
                    in_seq,out_seq=seq[:i],seq[i]
                    in_seq=pad_sequences([in_seq],maxlen=max_length)[0]
                    
                    out_seq=to_categorical([out_seq],num_classes=vocab_size)[0]
                    
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
                    
            if n==num_photos_per_batch:
                yield [[array(X1),array(X2)],array(y)]
                X1,X2,y=list(),list(),list()
                n=0

In [35]:
glove_dir='glove.6B.200d.txt'
embedding_index={}

f=open(glove_dir,encoding="utf-8")
for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embedding_index[word]=coefs
f.close()

print("found %s word vectors",len(embedding_index))

found %s word vectors 400000


In [36]:
embedding_dim=200
embedding_matrix=np.zeros((vocab_size,embedding_dim))

for word, i in wordtoix.items():
    embedding_vector=embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i]=embedding_vector

In [37]:
embedding_matrix.shape

(5436, 200)

In [38]:
inputs1=Input(shape=(2048,))
fe1=Dropout(0.5)(inputs1)
fe2=Dense(256,activation='relu')(fe1)
inputs2=Input(shape=(max_length,))

se1=Embedding(vocab_size,embedding_dim,mask_zero=True)(inputs2)
se2=Dropout(0.5)(se1)
se3=LSTM(256)(se2)
decoder1=add([fe2,se3])
decoder2=Dense(256,activation='relu')(decoder1)
outputs=Dense(vocab_size,activation='softmax')(decoder2)
model=Model(inputs=[inputs1,inputs2],outputs=outputs)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [39]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 72)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 72, 200)      1087200     input_3[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2048)         0           input_2[0][0]                    
____________________________________________________________________________________________

In [41]:
model.layers[2]

In [39]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable=False

In [40]:
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [41]:
epochs=9
number_pics_per_batch=3
steps=len(descriptions)//number_pics_per_batch

In [42]:
del descriptions['image_name']
del descriptions['']

In [43]:
 model.save('./model_weights/model_'+str(0)+'.h5')

In [44]:
for i in range(epochs+1):
    generator=data_generator(descriptions,train_features,wordtoix,max_length,number_pics_per_batch)
    model.fit_generator(generator,epochs=1,steps_per_epoch=steps,verbose=1)
    model.save('./model_weights/model_'+str(i)+'.h5')


Epoch 1/1
10595/10595 [==============================] - 1085s 102ms/step - loss: 4.4316
Epoch 1/1
10595/10595 [==============================] - 1107s 105ms/step - loss: 3.9082
Epoch 1/1
10595/10595 [==============================] - 1121s 106ms/step - loss: 3.7857
Epoch 1/1
10595/10595 [==============================] - 1124s 106ms/step - loss: 3.7263
Epoch 1/1
10595/10595 [==============================] - 1123s 106ms/step - loss: 3.6953
Epoch 1/1
10595/10595 [==============================] - 1118s 106ms/step - loss: 3.6755
Epoch 1/1
10595/10595 [==============================] - 1114s 105ms/step - loss: 3.6632
Epoch 1/1
10595/10595 [==============================] - 1115s 105ms/step - loss: 3.6542
Epoch 1/1
  130/10595 [..............................] - ETA: 17:40 - loss: 3.6080

KeyboardInterrupt: 

In [48]:
import tensorflow as tf
new_model=tf.keras.models.load_model('./model_weights/model_7.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [53]:
model.compile(loss='categorical_crossentropy',optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=0.0001))
epochs = 10
number_pics_per_bath = 6
steps = len(descriptions)//number_pics_per_bath

In [54]:
for i in range(epochs+1):
    generator=data_generator(descriptions,train_features,wordtoix,max_length,number_pics_per_batch)
    model.fit_generator(generator,epochs=1,steps_per_epoch=steps,verbose=1)
    model.save('./model_weights/model_'+str(i+10)+'.h5')



Epoch 1/1
5297/5297 [==============================] - 531s 100ms/step - loss: 3.5347
Epoch 1/1
   1/5297 [..............................] - ETA: 10:39 - loss: 3.4657

c:\users\rahul\appdata\local\programs\python\python36\lib\site-packages\keras\engine\saving.py:165: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


5297/5297 [==============================] - 577s 109ms/step - loss: 3.4874
Epoch 1/1
5297/5297 [==============================] - 562s 106ms/step - loss: 3.4649
Epoch 1/1
5297/5297 [==============================] - 560s 106ms/step - loss: 3.4478
Epoch 1/1
5297/5297 [==============================] - 552s 104ms/step - loss: 3.4326
Epoch 1/1
5297/5297 [==============================] - 555s 105ms/step - loss: 3.4191
Epoch 1/1
5297/5297 [==============================] - 551s 104ms/step - loss: 3.4078
Epoch 1/1
5297/5297 [==============================] - 556s 105ms/step - loss: 3.3965
Epoch 1/1
5297/5297 [==============================] - 576s 109ms/step - loss: 3.3878
Epoch 1/1
5297/5297 [==============================] - 582s 110ms/step - loss: 3.3789
Epoch 1/1
5297/5297 [==============================] - 583s 110ms/step - loss: 3.3708


In [40]:
import tensorflow as tf
model=tf.keras.models.load_model('./model_weights/final_model.h5')
model.compile(loss='categorical_crossentropy',optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=0.0001))


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [41]:
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [42]:
test_folder='./test/'
test_images=glob.glob(test_folder+'*.jpg')

In [43]:
start=time()
encoding_test={}
for img in test_images:
    encoding_test[img[len(test_folder):]]=encode(img)
    print("time tken in second=",time()-start)


time tken in second= 14.85458254814148
time tken in second= 14.965773582458496
time tken in second= 15.025423288345337
time tken in second= 15.100020408630371
time tken in second= 15.182687997817993


In [44]:
pics=list(encoding_test.keys())
for pic in pics:
    image=encoding_test[pic].reshape((1,2048))
    print("greedy:",greedySearch(image))
    print('\n')

greedy: students are sitting at table with computers and wineglasses and drinks and drinks and drinking beer and drinking beer and drinking beer and drinking beer and drinking beer and drinking beer and drinking beer and drinking beer and drinking beer and drinking beer and drinking beer and drinking beer and drinking beer and drinking beer and drinking beer and drinking beer and drinking beer and drinking beer and drinking beer and


greedy: is being held by man in blue shirt and blue jeans and woman in black shirt are standing in front of the building with their arms raised and legs spread open open and the other is holding up sign that says free to the camera and the man is standing on the ground with his hands on his hips and is holding up sign that says free to the man is


greedy: troop of people are standing in front of building with their backs turned in the air and one of them is wearing black shirt and the other is wearing black shirt and holding up sign that says free to the